In [3]:
import pandas as pd
#import urbanaccess as ua
import pandana as pdna
import osmnet as osm
#import urbanaccess.plot
import pandana.network
import time
from simpledbf import Dbf5
import sys
import math
import gmplot

SQLalchemy is not installed. No support for SQL output.


In [7]:
NET_PATH = './data/net_am_peak/final_net.h5'
DBF_PATH = "./data/tl_2010_06037_tabblock10/tl_2010_06037_tabblock10.dbf"
SAVE_FILE = "./data/net_am_peak/block_am_closest_node2.csv"

In [8]:
dbf = Dbf5(DBF_PATH);

In [9]:
data_db = dbf.to_dataframe();

In [10]:
#store = pd.HDFStore('./data/small_net.h5', "r")
store = pd.HDFStore(NET_PATH, "r")

In [11]:
nodes = store.nodes
edges = store.edges
net=pdna.Network(nodes["x"], nodes["y"], edges["from"], edges["to"],
                 edges[["weight"]])

In [12]:
census_block_db = data_db[['GEOID10', 'INTPTLAT10', 'INTPTLON10']];

In [13]:
census_block_db['INTPTLAT10'] = census_block_db.apply(lambda x : float(x['INTPTLAT10']), axis = 1)
census_block_db['INTPTLON10'] = census_block_db.apply(lambda x : float(x['INTPTLON10']), axis = 1)

In [14]:
#census_block_db['node_id'] = census_block_db.apply(lambda x : getClosestNode(x), axis = 1)
census_block_db['node_id'] = net.get_node_ids(census_block_db['INTPTLON10'],census_block_db['INTPTLAT10'])

In [15]:
census_block_db['net_lat'] = census_block_db.apply(lambda x : nodes.loc[x['node_id'], 'y'], axis = 1)
census_block_db['net_lon'] = census_block_db.apply(lambda x : nodes.loc[x['node_id'], 'x'], axis = 1)

In [16]:
census_block_db = census_block_db.set_index('GEOID10')

In [17]:
def outsideBox(geoid):
    lat = census_block_db.loc[geoid,'INTPTLAT10']
    lon = census_block_db.loc[geoid,'INTPTLON10']
    if lat < 33.8 or lat > 34.25 or lon < -118.7 or lon > -118.0: #big
    #if lat < 34.00 or lat > 34.06 or lon < -118.30 or lon > -118.25: #small
        return 0
    return 1

In [18]:
census_block_db['in_box'] = census_block_db.apply(lambda x : outsideBox(x.name), axis = 1)

In [19]:
census_block_sub = census_block_db[census_block_db.in_box == 1]

In [20]:
census_block_sub.to_csv(SAVE_FILE);

In [ ]:
#end

In [35]:
gmap = gmplot.GoogleMapPlotter.from_geocode('Los Angeles')

In [36]:
#gmap.scatter(nodes['y'].values.tolist(), nodes['x'].values.tolist())
gmap.scatter(census_block_sub['net_lat'].values.tolist(), census_block_sub['net_lon'].values.tolist(), 'r',marker=False)

In [33]:
for i in range(0, len(edges)):
    edge = edges.iloc[i]
    from_point = nodes.iloc[edge['from'] - 1]
    to_point = nodes.iloc[edge['to'] - 1]
    gmap.plot((from_point['y'], to_point['y']),(from_point['x'], to_point['x']))

In [37]:
gmap.draw("./data/map.html")

In [31]:
census_block_db

,INTPTLAT10,INTPTLON10,node_id,net_lat,net_lon,in_box
GEOID10,,,,,,
060373006004026,34.197078,-118.231718,104932,34.197488,-118.230563,1
060373106012020,34.191093,-118.326216,384283,34.191174,-118.325751,1
060373102023003,34.190033,-118.295874,19220,34.191326,-118.294945,1
060373007015000,34.202718,-118.237677,351205,34.204203,-118.237912,1
060373007015006,34.191512,-118.230884,52666,34.190716,-118.229702,1
060371278052004,34.191199,-118.461271,86780,34.190304,-118.460847,1
060371277122002,34.195727,-118.471103,126405,34.195321,-118.470555,1
060371279202007,34.196203,-118.437260,60962,34.194844,-118.437812,1
060371310202017,34.196664,-118.537958,35489,34.195683,-118.537324,1


In [33]:
census_block_sub

,INTPTLAT10,INTPTLON10,node_id,net_lat,net_lon,in_box
GEOID10,,,,,,
060373006004026,34.197078,-118.231718,104932,34.197488,-118.230563,1
060373106012020,34.191093,-118.326216,384283,34.191174,-118.325751,1
060373102023003,34.190033,-118.295874,19220,34.191326,-118.294945,1
060373007015000,34.202718,-118.237677,351205,34.204203,-118.237912,1
060373007015006,34.191512,-118.230884,52666,34.190716,-118.229702,1
060371278052004,34.191199,-118.461271,86780,34.190304,-118.460847,1
060371277122002,34.195727,-118.471103,126405,34.195321,-118.470555,1
060371279202007,34.196203,-118.437260,60962,34.194844,-118.437812,1
060371310202017,34.196664,-118.537958,35489,34.195683,-118.537324,1
